# Intensity adjustment

Since ```biobeam``` produces intensities << 1 , we adjust them to approximately reach a desired SNR 

In [ ]:
import warnings
import numpy as np
from numpy.random import poisson
from itertools import product
from sim_util import load_tiff_sequence, save_as_sequence

# what we need to load
n_tiles_x = 4
n_tiles_y = 4
n_tiles_z = 1
two_sided_illum = True

# quantiles to use for normailzation
q_signal = 0.98
q_descs = 1.0

# desired SNR ~ sqrt(intensity at quantile)
snr_signal = 20
snr_descs = 30

# in and out paths
# {x},{y},{z},{ill} will be filled with respective indices
in_fpath = '/Volumes/davidh-ssd/mv-sim/sim3_1/sim3_1-bbeam-x{x}-y{y}-z{z}-i{ill}-l500/'
out_fpath = '/Volumes/davidh-ssd/mv-sim/sim3_1/intensity_adjusted/sim3_1-bbeam-x{x}-y{y}-z{z}-i{ill}-l500/'

# should we add Poisson shot noise?
add_poisson = True

signal_imgs = []
desc_imgs = []
for (x,y,z,ill) in product(range(n_tiles_x), range(n_tiles_y), range(n_tiles_z), [1] if not two_sided_illum else [1,2]):

    signal_img = load_tiff_sequence(in_fpath.format(x=x, y=y, z=z, ill=ill), 'bbeam_sim_c1.*')
    desc_img = load_tiff_sequence(in_fpath.format(x=x, y=y, z=z, ill=ill), 'bbeam_sim_c2.*')
    signal_imgs.append(signal_img)
    desc_imgs.append(desc_img)
    
    print('loaded tile x={} y={}, z={}, illum={}'.format(x,y,z,ill))

# get mean quantile for all images
qs_signal = np.array([np.quantile(signal_img, q_signal) for signal_img in signal_imgs])
qs_desc = np.array([np.quantile(desc_img, q_descs) for desc_img in desc_imgs])
mq_signal = np.mean(qs_signal)
mq_desc = np.mean(qs_desc)

print('mean {}-quantile of signal images: {}'.format(q_signal, mq_signal))
print('mean {}-quantile of descriptor images: {}'.format(q_descs, mq_desc))

for (i,(x,y,z,ill)) in enumerate(product(range(n_tiles_x), range(n_tiles_y), range(n_tiles_z), [1] if not two_sided_illum else [1,2])):
    
    signal_img = signal_imgs[i]
    desc_img = desc_imgs[i]
    
    # normalize and scale intensity to desired SNR
    signal_img /= mq_signal
    desc_img /= mq_desc
    signal_img *= snr_signal**2
    desc_img *= snr_descs**2
    
    # shot noise
    if add_poisson:
        signal_img = poisson(signal_img)
        desc_img = poisson(desc_img)
        
    out_dir = out_fpath.format(x=x, y=y, z=z, ill=ill)
    with warnings.catch_warnings():
        # ignore annoying "low constrast image" warnings printed by skimage
        warnings.filterwarnings("ignore", message=".*?low contrast image.*?")
        save_as_sequence(signal_img, out_dir, 'bbeam_sim_c1_z{plane}.tif')
        save_as_sequence(desc_img, out_dir, 'bbeam_sim_c2_z{plane}.tif')
    
    print('saved intensity adjusted tile x={} y={}, z={}, illum={}'.format(x,y,z,ill))
    
